In [11]:
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-chroma 

Note: you may need to restart the kernel to use updated packages.


In [15]:
%pip install --upgrade --quiet langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [15]:
!pip install -qU langchain-google-vertexai

In [1]:
!pip install pypdf

Defaulting to user installation because normal site-packages is not writeable


In [1]:
!pip install -qU langchain-google-vertexai

In [7]:
!pip install transformers==4.39.0

Defaulting to user installation because normal site-packages is not writeable
  Using cached transformers-4.39.0-py3-none-any.whl (8.8 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 1.8 MB/s eta 0:00:00m eta 0:00:010:01:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 1.6 MB/s eta 0:00:000:00:010:00:01:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1


In [3]:
from dotenv import load_dotenv
import os
load_dotenv()

openai_api_key = os.getenv('openai_api_key')
google_api_key = os.getenv("GOOGLE_API_KEY")

# 1. Indexing: Load

In [32]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("../data/loi.pdf")
data = loader.load()

In [33]:
len(data[0].page_content[:])

33

# 2. Indexing: Split

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(data)

In [13]:
len(all_splits)

444

# 3. Indexing: Store

In [7]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings(api_key=openai_api_key))

# 4. Retrieval

In [8]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})
retrieved_docs = retriever.invoke("القانون من خالف الضوء الاحمر")

In [11]:
len(retrieved_docs)

6

In [9]:
print(retrieved_docs[0].page_content)

6. إذا ارتكب إحدى المخالفات التالية : 
أ. عدم احترام الوقوف الإجباري المفروض بضوء تشوير احمر  ؛  
ب. عدم احترام الوقوف الإجباري المفروض بعلامة "قف" Stop) ؛ 
ج. عدم احترام حق الأسبقية  ؛  
د. التوقف غير القانوني ليلا ومن غير إنارة خارج تجمع عمراني   ؛  
ه. عدم توفر المركبة على الح صارات المحددة بالنصوص التنظيمية  ؛  
و. السير في الاتجاه الممنوع؛  
 
138- أنظر  المادة  27 من المرسوم رقم2.10.311، السالف الذكر 
المادة 27 
"  لا يترتب استرجاع النقط على إلزامية الخضوع لدورة التربية على السلامة الطرقية التي يخضع لها صاحب  
رخصة السياقة، تطبيقا لأحكام المواد 168 و170 و173 من القانون رقم52.05  السالف الذكر 
139 - تم تغيير وتتميم المادة169  أعلاه،  بمقتضى المادة الأولى من القانون رقم116.14، السالف الذكر


# 5. Generation

In [15]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125",openai_api_key=openai_api_key)

In [16]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [17]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [18]:
rag_chain.invoke("القانون من خالف الضوء الاحمر")

'لا يترتب استرجاع النقط على إلزامية الخضوع لدورة التربية على السلامة الطرقية في حالة خرق مثل عدم احترام الوقوف الإجباري المفروض بضوء تشوير أحمر. المادة 27 من المرسوم رقم 2.10.311 تحدد العقوبات والإجراءات المتخذة في حالات المخالفات لقواعد السير على الطرق. توجه النيابة العامة المختصة إلى المصالح الجهوية أو الإقليمية لتبليغ القرارات القضائية داخل أجل 15 يوما.'

DONE